In [9]:
# Instalação das bibliotecas
!pip install transformers torch sentencepiece pandas accelerate bitsandbytes

# Atualização do transformers para garantir compatibilidade
!pip install --upgrade transformers

# Necessário para usar o Phi-3
!pip install 'optimum[onnxruntime]'

import sys
import os
import re
import json
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

# Garante a codificação UTF-8
try:
    sys.stdout.reconfigure(encoding='utf-8')
except Exception:
    pass

print("Setup de bibliotecas concluído.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 51.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.0
    Uninstalling transformers-4.57.0:
      Successfully uninstalled transformers-4.57.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.3/162.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 130.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 123.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.1/192.1 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.3 MB/s eta 0:00:00
^C
Setup de bibliotecas concluído.


In [4]:
# Importações repetidas aqui por segurança, caso o Colab não mantenha o namespace global
import sys
import os
import json
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# (As outras importações serão carregadas na próxima célula)


# =========================================================================
# 1. CARREGAMENTO DOS DADOS LOCAIS (COM O CAMINHO CORRIGIDO)
# =========================================================================

# **CAMINHO AJUSTADO:** Usando o caminho exato /content/data/train.json
DATA_FILE_PATH = "/content/data/train.json"
try:
    print(f"Iniciando o carregamento manual do JSON no ambiente Colab: {DATA_FILE_PATH}...")

    # Criamos o diretório de dados para que o arquivo possa ser movido/encontrado lá
    OUTPUT_DATA_DIR = os.path.dirname(DATA_FILE_PATH)
    os.makedirs(OUTPUT_DATA_DIR, exist_ok=True)

    # Verifica se o arquivo está na raiz e o move, se necessário
    if os.path.exists("/content/train.json") and not os.path.exists(DATA_FILE_PATH):
        os.rename("/content/train.json", DATA_FILE_PATH)
        print("Arquivo 'train.json' movido da raiz para /content/data/.")


    with open(DATA_FILE_PATH, 'r', encoding='utf-8') as f:
        # Lê o formato JSON Lines (JSONL)
        data_lines = [json.loads(line) for line in f]

    if not data_lines:
        raise ValueError("O arquivo JSON está vazio ou o formato está incorreto.")

    example_row = data_lines[0]

    context = example_row.get('content', 'ERRO: Chave content Ausente').strip()
    expected_answer = example_row.get('answer', 'ERRO: Chave answer Ausente').strip()

    print(f"\n✅ SUCESSO! Dataset carregado do arquivo local do Colab.")
    print(f"Exemplo de contexto (content): {context[:100]}...")
    print(f"Resposta de referência (answer): {expected_answer}")

except (FileNotFoundError, ValueError, Exception) as e:
    print(f"\n❌ ERRO FATAL: Falha no carregamento dos dados locais no Colab.")
    print(f"Erro: {e}")
    sys.exit() # sys.exit é seguro aqui pois sys foi importado


# =========================================================================
# VARIÁVEIS GLOBAIS DE DADOS (USADAS NA PRÓXIMA CÉLULA)
# =========================================================================

TARGET_INFERENCE = "pronominal bridging"

# Estas variáveis agora são definidas e podem ser usadas na Célula 3
# context e expected_answer

Iniciando o carregamento manual do JSON no ambiente Colab: /content/data/train.json...

✅ SUCESSO! Dataset carregado do arquivo local do Colab.
Exemplo de contexto (content): once upon a time there was a king who went forth into the world and fetched back a beautiful queen ....
Resposta de referência (answer): the people wished their king all that was good .


In [6]:
# CÉLULA 3: Carregamento dos Modelos (Phi-3 e BERT) - CORRIGIDA

# =========================================================================
# 3. CARREGAMENTO DE MODELOS (PHI-3 e BERT)
# =========================================================================

# Importações Essenciais (Repetidas por segurança do escopo no Colab)
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

# Ajuste para usar a GPU (se disponível) ou CPU.
device = "cuda:0" if torch.cuda.is_available() else "cpu"
# Usamos float16 para economia de VRAM (essencial para o Phi-3)
MODEL_DTYPE = torch.float16 if torch.cuda.is_available() else torch.float32

print(f"\nCarregando gerador de perguntas (PHI-3-mini-4k-instruct) para dispositivo: {device}...")
qg_model_id = "microsoft/Phi-3-mini-4k-instruct"

try:
    # Phi-3 usa o pipeline 'text-generation'
    qg = pipeline(
        "text-generation",
        model=qg_model_id,
        device=device,
        model_kwargs={"torch_dtype": MODEL_DTYPE}
    )
except Exception as e:
    print(f"❌ ERRO ao carregar o modelo Phi-3: {e}")
    sys.exit() # sys.exit é seguro pois sys foi importado na Célula 1


print("Carregando classificador de skills (Modelo BERT)...")
cls_model_id = "curious008/BertForStorySkillClassification"
cls_tokenizer = AutoTokenizer.from_pretrained(cls_model_id, use_fast=False)
cls_model = AutoModelForSequenceClassification.from_pretrained(
    cls_model_id, ignore_mismatched_sizes=True
)
if torch.cuda.is_available():
    cls_model.to("cuda")

skill_labels = [
    "Character", "Setting", "Feeling", "Action",
    "Causal Relationship", "Outcome Resolution", "Prediction"
]

print("Carregamento de modelos concluído.")


Carregando gerador de perguntas (PHI-3-mini-4k-instruct) para dispositivo: cuda:0...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!
`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at curious008/BertForStorySkillClassification and are newly initialized because the shapes did not match:
- bert.embeddings.word_embeddings.weight: found shape torch.Size([30524, 768]) in the checkpoint and torch.Size([30523, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Carregamento de modelos concluído.


In [11]:
# CÉLULA 4: Geração Controlada (QG), Classificação e Salvamento - VERSÃO FINAL COM PROMPT DE SAÍDA

# (Este bloco assume que as variáveis de contexto e modelos foram carregadas nas Células 2 e 3)

# =========================================================================
# 2. DEFINIÇÃO DA TAXONOMIA DE INFERÊNCIA E PROMPT ESTRUTURADO (CoT)
# =========================================================================

TARGET_INFERENCE = "pronominal bridging"

# Template Few-Shot/CoT (Mantido)
FEW_SHOT_EXAMPLE_TEMPLATE = f"""
[INSTRUCTION]
TASK: Generate a multiple-choice question for '{TARGET_INFERENCE}' inference type.
RULES: The output must adhere strictly to the format: NEW_TYPE, REASONING, QUESTION, OPTIONS.

Example:
TYPE: pronominal bridging
CONTEXT: A greenhouse is a building where plants such as flowers and vegetables are grown. It usually has a glass or translucent plastic roof.
ANSWER: greenhouses
REASONING: The pronoun 'it' refers to 'greenhouse' in the previous sentence, bridging the concepts.
QUESTION: According to the passage, what can have translucent plastic roofs?
OPTIONS: backyards; living spaces; greenhouses; botanic gardens

NEW_CONTEXT: {{context}}
NEW_ANSWER: {{answer}}
NEW_TYPE: pronominal bridging
REASONING: (Generate the reasoning for the new question here)
QUESTION: (Generate the question stem here)
OPTIONS: (Generate 4 options, separated by semicolons)
"""

def create_phi3_prompt_for_inference_type(context, answer, template):
    """Cria o prompt Few-Shot."""
    prompt = template.replace("{{context}}", context) \
                     .replace("{{answer}}", answer)
    # ** Aplica a formatação de chat para LLMs instrucionais **
    return f"<|user|>{prompt}<|end|><|assistant|>"

# =========================================================================
# 4. GERAÇÃO E AVALIAÇÃO (FLUXO METODOLÓGICO)
# =========================================================================

print("\n=== REPLICAÇÃO DA METODOLOGIA DO ARTIGO (QG PHI-3 e CLASSIFICAÇÃO) ===")
print(f"Tipo de Inferência ALVO: {TARGET_INFERENCE}")

print(f"\n1. Geração Controlada (QG) via Few-Shot/CoT (PHI-3)...")

# CRIAÇÃO FINAL DO INPUT PROMPT (A ser incluído no output)
qg_input = create_phi3_prompt_for_inference_type(
    context, expected_answer, FEW_SHOT_EXAMPLE_TEMPLATE
)

qg_output_list = qg(
    qg_input,
    max_new_tokens=512,
    do_sample=False,
    return_full_text=False
)
generated_output_full = qg_output_list[0]['generated_text'].strip()

print("\n--- Output Bruto Gerado (Simulando CoT com Phi-3) ---")
print(generated_output_full)


# Processar a saída (Parsing)
try:
    reasoning_match = re.search(r"REASONING:\s*(.*?)(?=\s*QUESTION:|$)", generated_output_full, re.DOTALL | re.IGNORECASE)
    question_match = re.search(r"QUESTION:\s*(.*?)(?=\s*OPTIONS:|$)", generated_output_full, re.DOTALL | re.IGNORECASE)

    reasoning = reasoning_match.group(1).strip() if reasoning_match else "ERRO: Parsing do Raciocínio Falhou."
    generated_q = question_match.group(1).strip() if question_match else "ERRO: Parsing da Pergunta Falhou."

    if "ERRO" in generated_q:
        generated_q = generated_output_full.strip()
        reasoning = "Parsing falhou; usando output completo como pergunta."

except Exception:
    generated_q = generated_output_full.strip()
    reasoning = "Parsing Geral Falhou."


# 5. Classificação
print("\n2. Avaliação da Pergunta Gerada (Classificador BERT)")

enc = cls_tokenizer(generated_q, return_tensors="pt", truncation=True, padding=True, max_length=512)
if torch.cuda.is_available():
    enc = {k:v.to("cuda") for k,v in enc.items()}
logits = cls_model(**enc).logits
probs = torch.softmax(logits, dim=-1).cpu().detach().numpy().flatten()
pred_idx = int(probs.argmax())
pred_label = skill_labels[pred_idx]

print("Pergunta Gerada (Stem - RQ1):", generated_q)
print("Raciocínio Gerado (CoT - RQ3):", reasoning)
print(f"\nPredição do Classificador (Skill Real - RQ2): {pred_label}")

print("Probabilidades por classe:")
for lbl, p in zip(skill_labels, probs):
    print(f"  {lbl:20s} {p:.4f}")


# =========================================================================
# 5. SALVAMENTO DOS OUTPUTS
# =========================================================================

# Salvamento na pasta /content/output do Colab
OUTPUT_DIR = "/content/output"
OUTPUT_FILENAME = "example_qg_classification_result_phi3.json"
outpath = os.path.join(OUTPUT_DIR, OUTPUT_FILENAME)

os.makedirs(OUTPUT_DIR, exist_ok=True)

results = {
    "target_inference_type": TARGET_INFERENCE,
    "context_snippet": context,
    "expected_answer": expected_answer,

    "input_prompt_full": qg_input,  # <--- NOVA VARIÁVEL ADICIONADA
    "generated_question_stem_rq1": generated_q,
    "generated_reasoning_cot_rq3": reasoning,

    "classification_model": cls_model_id,
    "predicted_skill_label_rq2": pred_label,
    "probabilities_by_class": {lbl: float(p) for lbl, p in zip(skill_labels, probs)}
}

try:
    with open(outpath, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    print(f"\nResultado salvo com sucesso em: {outpath}")

except Exception as e:
    print(f"\n❌ ERRO ao salvar o arquivo JSON.")
    print(f"Erro: {e}")


=== REPLICAÇÃO DA METODOLOGIA DO ARTIGO (QG PHI-3 e CLASSIFICAÇÃO) ===
Tipo de Inferência ALVO: pronominal bridging

1. Geração Controlada (QG) via Few-Shot/CoT (PHI-3)...

--- Output Bruto Gerado (Simulando CoT com Phi-3) ---
NEW_CONTEXT: A smartphone is a mobile device that allows users to make calls, send messages, and access the internet. It typically has a touchscreen interface.
NEW_ANSWER: smartphones
REASONING: The pronoun 'it' refers to'smartphone' in the previous sentence, bridging the concepts.
QUESTION: According to the passage, what typically has a touchscreen interface?
OPTIONS: tablets; desktop computers; smartphones; e-readers

2. Avaliação da Pergunta Gerada (Classificador BERT)
Pergunta Gerada (Stem - RQ1): NEW_CONTEXT: A smartphone is a mobile device that allows users to make calls, send messages, and access the internet. It typically has a touchscreen interface.
NEW_ANSWER: smartphones
REASONING: The pronoun 'it' refers to'smartphone' in the previous sentence, bridg